In [1]:
# selenium의 webdriver를 사용하기 위한 import
from selenium import webdriver

# selenium으로 키를 조작하기 위한 import
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

# 페이지 로딩을 기다리는데에 사용할 time 모듈 import
import time

# Open browser

In [34]:
# 크롬드라이버 실행. 4.6 버전부터 크롬드라이버 별도 다운로드 및 경로 설정 불필요
driver = webdriver.Chrome() 

#크롬 드라이버에 url 주소 넣고 실행
site = 'https://www.vivino.com/casillero-del-diablo-cabernet-sauvignon-reserva-central-valley/w/1135067?ref=nav-search#all_reviews'
driver.get(site)

# 페이지가 완전히 로딩되도록 3초동안 기다림
time.sleep(3)

# Get community review list

In [35]:
# Open community review window by clicking Show more reviews
path = "//button[@tabindex='0']"
#path = "//span[text()='Show more reviews']" # successful
#path = '//*[@id="all_reviews"]/div[3]/div[1]/button' # NoSuchElementException

driver.find_element(By.XPATH, path).click()
time.sleep(1)

# Show reviews by recent
path = '//*[@data-testid="baseModalBackdrop"]/div[2]/div[2]/div[2]/div/a[1]'

driver.find_element(By.XPATH, path).click()
time.sleep(1)

# Get reviews

In [94]:
class_review = "communityReview__reviewText--2bfLj"
class_date = "anchor_anchor__m8Qi- reviewAnchor__anchor--2NKFw reviewDate__reviewDate--49vpM undefined"
class_date = class_date.replace(' ', '.')
   

max_rev = 100
max_scr = 5
time_scr = 2

def extract_text(class_name, driver=driver, by=By.CLASS_NAME):
    l = driver.find_elements(by, class_name)
    return [x.text for x in l]


reviews = list()
dates = list()
n_scr = 0

while True:
    list_r = extract_text(class_review)
    list_d = extract_text(class_date)

    n = len(list_r) - len(reviews)
    if n > 0: # get new n reviews
        reviews.extend(list_r[-n:])
        dates.extend(list_d[-n:])
        n_scr = 0
    else:
        n_scr += 1

    if ((len(reviews) > max_rev) or (n_scr > max_scr)):
        break
    else:
        driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.PAGE_DOWN)
        time.sleep(time_scr)

#len(reviews), len(dates)

# close browser
driver.close()

# Save reviews

In [95]:
import pandas as pd

df_reviews = pd.DataFrame.from_dict({'date':dates, 'review':reviews})
df_reviews['date'] = pd.to_datetime(df_reviews['date'])
df_reviews.head()

,date,review
0,2024-01-24,Little too cherry on the front end for me
1,2024-01-20,"En liten skarp knekk i smaken. Ok fredagsvin,m..."
2,2024-01-19,Aight
3,2024-01-18,무난한 맛 가성비 좋은듯
4,2024-01-17,oak cherry black cherry chocolate blackcurrant...


In [84]:
f = 'wine_reviews'
df_reviews.to_csv(f, index=False)  

# Module

In [44]:
# selenium의 webdriver를 사용하기 위한 import
from selenium import webdriver

# selenium으로 키를 조작하기 위한 import
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

from selenium.common.exceptions import ElementNotVisibleException, StaleElementReferenceException, NoSuchElementException

# 페이지 로딩을 기다리는데에 사용할 time 모듈 import
import time

import pandas as pd 
import numpy as np
from tqdm import tqdm
from datetime import datetime

In [59]:
def collect_reviews(class_name, driver, by=By.CLASS_NAME):
    try:
        l = driver.find_elements(by, class_name)
        return [x.text for x in l]
    except:
        print(f'ERROR) Check locator: {class_name}')
        driver.close()
        return False


def click_locator(driver, *args, **kwargs):
    try:
        driver.find_element(*args, **kwargs).click()
        return True
    #except NoSuchElementException:
    except:
        print(f'ERROR) Check locator: {args[1]}')
        driver.close()
        return False


def vivino_reviews(wine_url, wine_name, 
                   end_date = '20230101',
                   max_rev = 1e4, 
                   max_scr = 10, # max num of scroll for new review list 
                   time_scr = 3, # time to wait after scrolling. StaleElementReferenceException if not enough
                   loc1=None, 
                   loc2=None, 
                   loc3=None, 
                   loc4=None,
                   # take the final review list as the review page has same old reiviews at the end every update by scroll-down 
                   final_only = True,
                   check_idx = 4, # community review list has 3 old reviews at the end of list every loading of new reviews
                   rev_date_format = '%b %d, %Y'
                  ):
    driver = webdriver.Chrome() 
    driver.get(wine_url)
    time.sleep(3)

    # enter community reviews
    if not click_locator(driver, By.XPATH, loc1):
        return None
    time.sleep(1)
    
    # Show reviews by recent
    if not click_locator(driver, By.XPATH, loc2):
        return None
    time.sleep(1)

    # collect reviews
    loc3 = loc3.replace(' ', '.')
    loc4 = loc4.replace(' ', '.')
     
    reviews = list()
    dates = list()
    n_scr = 0

    #pbar = tqdm(total=max_rev, position=0)
    pbar = tqdm(position=0)

    list_r = collect_reviews(loc3, driver)
    if not list_r:
        return None
    
    list_d = collect_reviews(loc4, driver)
    if not list_d:
        return None
    
    while True:
        n = len(list_r) - len(reviews)
        if n > 0: # get new n reviews
            if final_only: # replace review list with the latest scan
                reviews = list_r
                dates = list_d
            else: # update review list with new reviews from the latest scan
                reviews.extend(list_r[-n:])
                dates.extend(list_d[-n:])
            n_scr = 0
            pbar.update(n)
        else:
            n_scr += 1

        # testing
        #return list_d

        d1 = datetime.strptime(list_d[-check_idx], rev_date_format)
        d2 = datetime.strptime(end_date, '%Y%m%d')
        #print('testing:', d1)

        if ((len(reviews) > max_rev) or (d1 < d2)):
            print(f'{len(reviews)} reviews collected.')
            break
        elif (n_scr > max_scr):
            # redundunt as n_try checking max_scr as well?
            print(f'WARNING: No additional reviews after {max_scr} reloading.')
            break
        else:
            driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.PAGE_DOWN)
            time.sleep(time_scr)

            failed = True
            n_try = 0
            while failed:
                try:
                    list_r = collect_reviews(loc3, driver)
                    list_d = collect_reviews(loc4, driver)
                    failed = False
                except StaleElementReferenceException:
                    time.sleep(time_scr)
                    n_try += 1
                    
                if n_try > max_scr:
                    failed = False
                    print('WARNING: fail to collect all reviews')

    pbar.close()

    # close browser
    driver.close()

    # save result
    df_reviews = pd.DataFrame.from_dict({'date':dates, 'review':reviews})
    df_reviews['date'] = pd.to_datetime(df_reviews['date'])
    
    if False: #deprecated
        df_reviews.to_csv(filename, index=False)  

    return df_reviews


def generate_wine_id(df_reviews):
    if df_reviews.empty:
        wid = 0
    else:
        wid = df_reviews.id.max()
        wid = 0 if wid is np.nan else wid+1 # check if no data row
    return wid


def concat_reviews(df_reviews, df, col_rev, save=True):
    wine_id = generate_wine_id(df_reviews)
    df[['id', 'wine']] = [wine_id, wine_name]
    df = df.reindex(columns=col_rev)
    if save:
        df.to_csv(f'wine_{wine_id}.csv', index=False)
    
    df_reviews = pd.concat([df_reviews if not df_reviews.empty else None, df])
    return df_reviews

In [28]:
locator_more = "//button[@tabindex='0']"
locator_recent = '//*[@data-testid="baseModalBackdrop"]/div[2]/div[2]/div[2]/div/a[1]'

locator_review = "communityReview__reviewText--2bfLj"
locator_date = "anchor_anchor__m8Qi- reviewAnchor__anchor--2NKFw reviewDate__reviewDate--49vpM undefined"

kwargs = {'loc1': locator_more,
          'loc2': locator_recent, 
          'loc3': locator_review,
          'loc4': locator_date
}

In [60]:
kwargs.update({
    'end_date': '20230801',
    #'max_rev': 100, 
    'max_scr': 10, 'time_scr': 2,
})

df_reviews = pd.DataFrame()
col_rev = ['id', 'wine', 'date', 'review']

In [30]:
# testing

#kwargs.update({'loc1': locator_date})

wine_name = 'Casillero del Diablo Cabernet Sauvignon (Reserva)'
wine_url = 'https://www.vivino.com/casillero-del-diablo-cabernet-sauvignon-reserva-central-valley/w/1135067?ref=nav-search#all_reviews'

list_d = vivino_reviews(wine_url, wine_name, **kwargs)



 13%|████████                                                      | 13/100 [00:00<00:01, 62.72it/s]


In [43]:
from datetime import datetime

format_specifier = '%b %d, %Y'
datetime.strptime(list_d[-4], format_specifier) < datetime.strptime('20250101', '%Y%m%d')

True

In [42]:
list_d[-4]
#datetime.strptime('20230101', '%Y%m%d')

'Jan 8, 2024'

In [61]:
%%time

wine_name = 'Casillero del Diablo Cabernet Sauvignon (Reserva)'
wine_url = 'https://www.vivino.com/casillero-del-diablo-cabernet-sauvignon-reserva-central-valley/w/1135067?ref=nav-search#all_reviews'

df = vivino_reviews(wine_url, wine_name, **kwargs)

df_reviews = concat_reviews(df_reviews, df, col_rev, save=True)

33it [00:18,  6.04it/s]

ERROR) Check locator: communityReview__reviewText--2bfLj
ERROR) Check locator: anchor_anchor__m8Qi-.reviewAnchor__anchor--2NKFw.reviewDate__reviewDate--49vpM.undefined


InvalidSessionIdException: Message: invalid session id
Stacktrace:
#0 0x55e3cc10dd93 <unknown>
#1 0x55e3cbdf1177 <unknown>
#2 0x55e3cbe2ea5b <unknown>
#3 0x55e3cbe5f324 <unknown>
#4 0x55e3cbe59f50 <unknown>
#5 0x55e3cbe597ab <unknown>
#6 0x55e3cbdbd528 <unknown>
#7 0x55e3cc0d1e4b <unknown>
#8 0x55e3cc0d5dfa <unknown>
#9 0x55e3cc0be6d5 <unknown>
#10 0x55e3cc0d6a6f <unknown>
#11 0x55e3cc0a269f <unknown>
#12 0x55e3cbdbb8da <unknown>
#13 0x7f922ba29d90 <unknown>


In [58]:
df_reviews.tail()

,id,wine,date,review
108,1,Casillero del Diablo Cabernet Sauvignon (Reserva),2023-05-23,가성비로 무난 라이트함
109,1,Casillero del Diablo Cabernet Sauvignon (Reserva),2023-05-23,Ótimo valor equiparado ao que oferece. Custo b...
110,1,Casillero del Diablo Cabernet Sauvignon (Reserva),NaT,
111,1,Casillero del Diablo Cabernet Sauvignon (Reserva),2017-05-29,A decent Chilean cab served at the VIP lounge ...
112,1,Casillero del Diablo Cabernet Sauvignon (Reserva),2020-02-02,One if the most popular Chilean wines this wel...


In [213]:
wine_name = 'Yellow Tail Cabernet Sauvignon'
wine_url = 'https://www.vivino.com/yellow-tail-cabernet-sauvignon/w/2549?ref=nav-search#all_reviews'

df = vivino_reviews(wine_url, wine_name, **kwargs)

df_reviews = concat_reviews(df_reviews, df, col_rev, save=True)

113it [02:26,  1.30s/it]                                                                            


In [215]:
df_reviews.set_index('id')

,wine,date,review
id,,,
0,Casillero del Diablo Cabernet Sauvignon (Reserva),2024-01-24,Little too cherry on the front end for me
0,Casillero del Diablo Cabernet Sauvignon (Reserva),2024-01-20,"En liten skarp knekk i smaken. Ok fredagsvin,m..."
0,Casillero del Diablo Cabernet Sauvignon (Reserva),2024-01-19,Aight
0,Casillero del Diablo Cabernet Sauvignon (Reserva),2024-01-18,무난한 맛 가성비 좋은듯
0,Casillero del Diablo Cabernet Sauvignon (Reserva),2024-01-17,oak cherry black cherry chocolate blackcurrant...
...,...,...,...
1,Yellow Tail Cabernet Sauvignon,2022-04-23,I liked the flavor of berries and the sweetnes...
1,Yellow Tail Cabernet Sauvignon,2022-04-22,"Excellent for its value, bold, notes of blackb..."
1,Yellow Tail Cabernet Sauvignon,2021-05-23,


In [216]:
!zip wine_reviews.zip wine*csv

  adding: wine_0.csv (deflated 67%)
  adding: wine_1.csv (deflated 57%)


# lab

## locator check

In [5]:
from selenium.common.exceptions import NoSuchElementException

path = '//*[@data-testid="baseModalBackdrop"]/div[2]/div[2]/div[2]/div/a[10]'

try:
    driver.find_element(By.XPATH, path).click()
except NoSuchElementException:
    print(f'Check locator: {path}')
    
time.sleep(1)

Check locator: //*[@data-testid="baseModalBackdrop"]/div[2]/div[2]/div[2]/div/a[10]


In [17]:
path = '//*[@data-testid="baseModalBackdrop"]/div[2]/div[2]/div[2]/div/a[1]'

def locator_check(func,  *args):
    def wrapper(*args):
        try:
            func(*args)
        except NoSuchElementException:
            print(f'Check locator: {args[1]}')
    return wrapper

find_elem = locator_check(driver.find_element)
find_elem(By.XPATH, path).click()

Check locator: //*[@data-testid="baseModalBackdrop"]/div[2]/div[2]/div[2]/div/a[1]


AttributeError: 'NoneType' object has no attribute 'click'

In [26]:
path = '//*[@data-testid="baseModalBackdrop"]/div[2]/div[2]/div[2]/div/a[1]'

def locator_check(func):
    def wrapper(*args):
        try:
            return func(*args)
        except NoSuchElementException:
            print(f'Check locator: {args[1]}')
    return wrapper

driver.find_element = locator_check(driver.find_element)
driver.find_element(By.XPATH, path)
driver.click()  

Check locator: //*[@data-testid="baseModalBackdrop"]/div[2]/div[2]/div[2]/div/a[1]


AttributeError: 'WebDriver' object has no attribute 'click'

In [27]:
def click_locator(*args, **kwargs):
    try:
        driver.find_element(*args, **kwargs).click()
    except NoSuchElementException:
        print(f'Check locator: {args[1]}')

path = '//*[@data-testid="baseModalBackdrop"]/div[2]/div[2]/div[2]/div/a[1]'
click_locator(By.XPATH, path)

Check locator: //*[@data-testid="baseModalBackdrop"]/div[2]/div[2]/div[2]/div/a[1]


AttributeError: 'NoneType' object has no attribute 'click'

In [47]:
# 크롬드라이버 실행. 4.6 버전부터 크롬드라이버 별도 다운로드 및 경로 설정 불필요
driver = webdriver.Chrome() 

#크롬 드라이버에 url 주소 넣고 실행
site = 'https://www.vivino.com/casillero-del-diablo-cabernet-sauvignon-reserva-central-valley/w/1135067?ref=nav-search#all_reviews'
driver.get(site)

# 페이지가 완전히 로딩되도록 3초동안 기다림
time.sleep(3)

In [43]:
def click_locator(*args, **kwargs):
    try:
        driver.find_element(*args, **kwargs).click()
    except NoSuchElementException:
        print(f'Check locator: {args[1]}')


In [48]:
# Open community review window by clicking Show more reviews
path = "//button[@tabindex='0']"
#path = "//span[text()='Show more reviews']" # successful
#path = '//*[@id="all_reviews"]/div[3]/div[1]/button' # NoSuchElementException

click_locator(By.XPATH, path)
time.sleep(1)

# Show reviews by recent
#path = '//*[@data-testid="baseModalBackdrop"]/div[2]/div[2]/div[2]/div/a[1]'
path = '//*[@id="all_reviews"]/div[3]/div[1]/button' # NoSuchElementException

click_locator(By.XPATH, path)
time.sleep(1)

Check locator: //*[@id="all_reviews"]/div[3]/div[1]/button


In [19]:
if not 'test':
    print('True')